<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
Автор материала: Мария Мансурова, аналитик-разработчик в команде Яндекс.Метрики 
    
Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Домашнее задание № 9. (демо)
## <center> Анализ временных рядов
    
**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1ijk4aFKY5plPiI8z3Mgi3i1Ln94VBY9SSt6xGIdVVFQ/).**

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import pandas as pd

print(__version__) # need 1.9.0 or greater

init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

3.1.1


## Подготавливаем данные

Для начала скачаем данные в `dataframe`. Сегодня будем предсказывать просмотры wiki-страницы [Machine Learning](https://en.wikipedia.org/wiki/Machine_learning). Данные я скачала с помощью библиотеки [Wikipediatrend](https://www.r-bloggers.com/using-wikipediatrend/) для `R`.

In [4]:
df = pd.read_csv('../../data/wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [5]:
df.shape

(383, 7)

In [6]:
df.date = pd.to_datetime(df.date)

In [7]:
plotly_df(df.set_index('date')[['count']])

## Предсказание с помощью Facebook Prophet

Для начала построим предсказание с помощью простой библиотеки `Facebook Prophet`. Для того, чтобы посмотреть на качество модели, отбросим из обучающей выборки последние 30 дней.

In [8]:
from fbprophet import Prophet

INFO:matplotlib.font_manager:font search path ['C:\\Users\\balth\\Anaconda3\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf', 'C:\\Users\\balth\\Anaconda3\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\afm', 'C:\\Users\\balth\\Anaconda3\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\pdfcorefonts']
INFO:matplotlib.font_manager:generated new fontManager


In [11]:
predictions = 30

df = df[['date', 'count']]
df.columns = ['ds', 'y']
df.tail()

,ds,y
382,2016-01-16,1644
381,2016-01-17,1836
376,2016-01-18,2983
375,2016-01-19,3389
372,2016-01-20,3559


In [12]:
train_df = df[:-predictions].copy()

In [8]:
%%time
model = Prophet()
model.fit(train_df)

C:\Program Files (x86)\Anaconda\lib\site-packages\numexpr\cpuinfo.py:109: DeprecationWarning:

invalid escape sequence \d

C:\Program Files (x86)\Anaconda\lib\site-packages\numexpr\cpuinfo.py:662: DeprecationWarning:

invalid escape sequence \s

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Wall time: 2.08 s


In [9]:
future = model.make_future_dataframe(periods=30)
future.tail()

,ds
378,2016-01-16
379,2016-01-17
380,2016-01-18
381,2016-01-19
382,2016-01-20


In [10]:
forecast = model.predict(future)
forecast.tail()

,ds,trend,trend_lower,trend_upper,yhat_lower,yhat_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yhat
378,2016-01-16,2980.409971,2962.573158,2999.480068,1670.780370,2539.997617,-861.707294,-861.707294,-861.707294,0.0,0.0,0.0,-861.707294,-861.707294,-861.707294,2118.702677
379,2016-01-17,2985.826305,2966.370258,3006.048132,1851.014938,2694.737922,-720.725103,-720.725103,-720.725103,0.0,0.0,0.0,-720.725103,-720.725103,-720.725103,2265.101202
380,2016-01-18,2991.242639,2970.094816,3012.830511,2845.846675,3698.038261,281.348266,281.348266,281.348266,0.0,0.0,0.0,281.348266,281.348266,281.348266,3272.590905
381,2016-01-19,2996.658973,2974.509789,3019.797708,3121.448950,3944.695971,541.447195,541.447195,541.447195,0.0,0.0,0.0,541.447195,541.447195,541.447195,3538.106169
382,2016-01-20,3002.075307,2978.227947,3026.885628,3042.442315,3822.513966,425.562942,425.562942,425.562942,0.0,0.0,0.0,425.562942,425.562942,425.562942,3427.638249


**Вопрос 1:** Какое предсказание числа просмотров wiki-страницы на 20 января? Ответ округлите до целого числа.

Оценим качество предсказания по последним 30 точкам.

In [21]:
import numpy as np

In [12]:
## ВАШ КОД для оценки качества полученной модели ##
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MAPE', np.mean(abs(cmp_df[-predictions:]['p'])))
print('MAE', np.mean(abs(cmp_df[-predictions:]['e'])))

MAPE 34.54741609421443
MAE 600.7629173456852


**Вопрос 2**: Какое получилось MAPE?

**Вопрос 3**: Какое получилось MAE?

## Предсказываем с помощью ARIMA

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\iolib\table.py:882: DeprecationWarning:

invalid escape sequence \#

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\iolib\table.py:882: DeprecationWarning:

invalid escape sequence \$

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\iolib\table.py:882: DeprecationWarning:

invalid escape sequence \%

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\iolib\table.py:882: DeprecationWarning:

invalid escape sequence \&

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\iolib\table.py:882: DeprecationWarning:

invalid escape sequence \_

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py:1051: DeprecationWarning:

invalid escape sequence \*

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\mlemodel.py:1174: DeprecationWarning:

invalid escape sequence \*

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\simulation_smoother.py:428: Deprecat

C:\Users\balth\Anaconda3\lib\site-packages\statsmodels\multivariate\factor_rotation\_gpa_rotation.py:63: DeprecationWarning:

invalid escape sequence \p



**Вопрос 4:** Проверим стационарность ряда с помощью критерия Дики-Фулера. Является ли ряд стационарным? Какое значение p-value?

In [13]:
## ВАШ КОД для проверки стационарности ряда ##
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(train_df['y'])[1])

Критерий Дики-Фуллера: p=0.107392


**Вопрос 5**: Далее перейдем к построению модели SARIMAX (`sm.tsa.statespace.SARIMAX`). Модель c какими параметрами лучшая по `AIC`-критерию?

In [14]:
train_df.set_index('ds', inplace=True)

In [15]:
train_df['y_diff'] = train_df.y - train_df.y.shift(7)

In [16]:
ps = range(0, 2)
ds = range(0, 2)
qs = range(0, 4)
Ps = range(0, 4)
Ds = range(0, 3)
Qs = range(0, 2)

In [17]:
from itertools import product

parameters = product(ps, ds, qs, Ps, Ds, Qs)
parameters_list = list(parameters)

In [22]:
%%time
import warnings
from tqdm import tqdm
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')

for param in tqdm(parameters_list):
    #try except нужен, потому что на некоторых наборах параметров модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(train_df['y'], order=(param[0], param[1], param[2]), 
                                        seasonal_order=(param[3], param[4], param[5], 7)).fit(disp=-1)
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except (ValueError, np.linalg.LinAlgError):
        continue
    aic = model.aic
    #сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

100%|████████████████████████████████████████████████████████████████████████████████| 384/384 [21:23<00:00,  3.34s/it]


Wall time: 21min 23s


In [26]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())

             parameters          aic
137  (0, 1, 2, 3, 2, 1)  4961.632632
271  (1, 1, 1, 3, 2, 1)  4962.857167
160  (0, 1, 3, 3, 2, 1)  4969.534606
181  (1, 0, 0, 3, 2, 1)  4973.212241
77   (0, 0, 3, 3, 2, 1)  4978.036900


In [27]:
print(best_model.summary())

                                 Statespace Model Results                                
Dep. Variable:                                 y   No. Observations:                  353
Model:             SARIMAX(0, 1, 2)x(3, 2, 1, 7)   Log Likelihood               -2473.816
Date:                           Wed, 15 Aug 2018   AIC                           4961.633
Time:                                   12:00:57   BIC                           4988.394
Sample:                                        0   HQIC                          4972.298
                                           - 353                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.5225      0.038    -13.806      0.000      -0.597      -0.448
ma.L2         -0.3271      0.044     -7.474